In [ ]:
%%capture
!pip install geopandas
!pip install sddk

In [ ]:
import matplotlib.pyplot as plt
import pandas as pd
import geopandas as gpd
import requests
from concurrent.futures import ThreadPoolExecutor
import time
import sddk

In [ ]:
s = sddk.cloudSession("sciencedata.dk", "SDAM_root", "648597@au.dk")

In [ ]:
# import the datasets...
PHI = pd.read_json("https://zenodo.org/record/4972968/files/PHI_v1.0.json?download=1")

# Minimal test

In [ ]:
PHI_sample = PHI.sample(1000, random_state=1)
PHI_sample.head(5)

In [ ]:
# trismegistos api
base_url = "https://www.trismegistos.org/dataservices/texrelations/{0}?source=phi"

In [ ]:
%%time
resps = []
for n in PHI_sample["PHI_ID"][:100]:
    resp_json = requests.get(base_url.format(str(n))).json()
    resp_json = dict([(list(el.keys())[0], list(el.values())[0]) for el in resp_json])
    resps.append(resp_json)

In [ ]:
PHI_sample_alts = pd.DataFrame.from_records(resps)
PHI_sample_alts.head(10)

# Main function (using paralel requests)

In [ ]:
def get_inscription_data(phi_id):
    try:
        resp_json = requests.get(base_url.format(str(phi_id))).json()
        resp_json = dict([(list(el.keys())[0], list(el.values())[0]) for el in resp_json])
    except:
        time.sleep(1)
        try:
            resp_json = requests.get(base_url.format(str(phi_id))).json()
            resp_json = dict([(list(el.keys())[0], list(el.values())[0]) for el in resp_json])
        except:
            resp_json = {}
    return resp_json

# Minimal experiment with paralels

In [ ]:
%%time
### main run of the function

step=100
all_inscriptions = []
for num in range(0, 300, step):
    actual_nums = PHI[num:num+step]["PHI_ID"].tolist()
    with ThreadPoolExecutor(max_workers=step*1.5) as pool:
        currently_parsed = list(pool.map(get_inscription_data,actual_nums))
    all_inscriptions.extend(currently_parsed)

In [ ]:

pd.DataFrame(all_inscriptions)

# Test with saving

In [ ]:
list(range(0, len(PHI), step))[-1]

In [ ]:
checkspoints = list(range(0, len(PHI), 1000))[1:] + [list(range(0, len(PHI), step))[-1]] # its own range plus last element of steps
print(checkspoints)

In [ ]:
%%time
### main run of the function

step=100
all_inscriptions = []
for num in range(0, 3001, step):
    actual_nums = PHI[num:num+step]["PHI_ID"].tolist()
    with ThreadPoolExecutor(max_workers=step*1.5) as pool:
        currently_parsed = list(pool.map(get_inscription_data,actual_nums))
    all_inscriptions.extend(currently_parsed)
    if num in checkspoints:
      s.write_file("SDAM_data/PHI/PHI_TM_{0}.csv".format(str(num)), pd.DataFrame(all_inscriptions))
      all_inscriptions = []

In [ ]:
PHI_TM_1000 = s.read_file("SDAM_data/PHI/PHI_TM_{0}.csv".format(str(1000)))
PHI_TM_2000 = s.read_file("SDAM_data/PHI/PHI_TM_{0}.csv".format(str(2000)))
PHI_TM_3000 = s.read_file("SDAM_data/PHI/PHI_TM_{0}.csv".format(str(3000)))

In [ ]:
PHI_TM_merge = pd.DataFrame()
for df in [PHI_TM_1000, PHI_TM_2000, PHI_TM_3000]:
  PHI_TM_merge = PHI_TM_merge.append(df)

In [ ]:
len(PHI_TM_merge)

# Main extraction

In [ ]:
step=100
checkspoints = list(range(0, len(PHI), 5000))[1:] + [list(range(0, len(PHI), step))[-1]] # its own range plus last element of steps
print(checkspoints)

all_inscriptions = []
for num in range(0, len(PHI), step):
    actual_nums = PHI[num:num+step]["PHI_ID"].tolist()
    with ThreadPoolExecutor(max_workers=step*1.5) as pool:
        currently_parsed = list(pool.map(get_inscription_data,actual_nums))
    all_inscriptions.extend(currently_parsed)
    if num in checkspoints:
      s.write_file("SDAM_data/PHI/PHI_TM_{0}.csv".format(str(num)), pd.DataFrame(all_inscriptions))
      all_inscriptions = []


# Merge files

In [ ]:
test = s.read_file("SDAM_data/PHI/PHI_TM_{0}.csv".format(str(80000)))
test.head(20)

In [ ]:
step = 100
checkpoints = list(range(0, len(PHI), 5000))[1:] + [list(range(0, len(PHI), step))[-1]] # its own range plus last element of steps

PHI_TM_merged = s.read_file("SDAM_data/PHI/PHI_TM_{0}.csv".format(str(checkpoints[0])))
for num in checkspoints[1:]:
  filename = "SDAM_data/PHI/PHI_TM_{0}.csv".format(str(num))
  PHI_TM_merged = PHI_TM_merged.append(s.read_file(filename))
  print(filename)

In [ ]:
len(PHI_TM_merged)

In [ ]:
PHI_TM_merged[100000:100010]

In [ ]:
s.write_file("SDAM_data/PHI/PHI_TM_merged.csv", PHI_TM_merged)

In [ ]:
#PHI_TM_merged = s.read_file("SDAM_data/PHI/PHI_TM_merged.csv")

In [ ]:
to_keep = []
for col in PHI_TM_merged.columns:
  length_vals = len([el for el in PHI_TM_merged[col] if "[" in str(el)])
  if length_vals >= 1000:
    to_keep.append(col)
    print(col, length_vals)

In [ ]:
PHI_TM_merged = PHI_TM_merged[to_keep]
PHI_TM_merged.head(10)

In [ ]:
def try_eval(val):
  if not isinstance(val, list):
    try: 
      val = eval(val)
    except:
      val = None
  return val

In [ ]:
for col in PHI_TM_merged.columns:
  PHI_TM_merged[col] = PHI_TM_merged[col].apply(try_eval)

In [ ]:
PHI_TM_merged.head(5)

In [ ]:
for col in PHI_TM_merged.columns:
  if "ID" not in col:
    PHI_TM_merged.rename(columns={col : col + "_ID"}, inplace=True)

In [ ]:
PHI_TM_merged.head(5)

In [ ]:
def get_first_id(val):
  try: 
    return int(val[0])
  except: 
    return val

PHI_TM_merged["PHI_ID"] = PHI_TM_merged["PHI_ID"].apply(get_first_id)

In [ ]:
def to_int(val):
  try: 
    return int64(val)
  except: 
    return val

PHI_TM_merged["PHI_ID"] = PHI_TM_merged["PHI_ID"].apply(to_int)

In [ ]:
PHI_TM_merged.head(5)

In [ ]:
PHI["PHI_ID"]

In [ ]:
PHI_TM_merged["PHI_ID"].astype(int)

In [ ]:
PHI.merge(PHI_TM_merged, on="PHI_ID")

In [ ]:
s.write_file("SDAM_data/PHI/PHI_TMids.json", PHI)

In [ ]:

pd.DataFrame(all_inscriptions)

,TM_ID,EDB,EDH,EDCS,EDR,HE,UOXF,RIB,PHI,LUPA,...,ALA,CPI,GLAUx,Papyrus_Projekt,GEM,Innovating_knowledge,PUL,4CARE/DEChriM,NYU_Amheida,WARNING
0,[786251],NaN,NaN,None,NaN,NaN,NaN,NaN,[1],NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,[786252],NaN,NaN,None,NaN,NaN,NaN,NaN,[2],NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,[786253],NaN,NaN,None,NaN,NaN,NaN,NaN,[3],NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,[786254],NaN,NaN,None,NaN,NaN,NaN,NaN,[4],NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,[786255],NaN,NaN,None,NaN,NaN,NaN,NaN,[5],NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
295,[786544],NaN,NaN,None,NaN,NaN,NaN,NaN,[296],NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
296,[786545],NaN,NaN,None,NaN,NaN,NaN,NaN,[297],NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
297,[786546],NaN,NaN,None,NaN,NaN,NaN,NaN,[298],NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
298,[786547],NaN,NaN,None,NaN,NaN,NaN,NaN,[299],NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


# Test with saving

In [ ]:
list(range(0, len(PHI), step))[-1]

In [ ]:
checkspoints = list(range(0, len(PHI), 1000))[1:] + [list(range(0, len(PHI), step))[-1]] # its own range plus last element of steps
print(checkspoints)

[1000, 2000, 3000, 4000, 5000, 6000, 7000, 8000, 9000, 10000, 11000, 12000, 13000, 14000, 15000, 16000, 17000, 18000, 19000, 20000, 21000, 22000, 23000, 24000, 25000, 26000, 27000, 28000, 29000, 30000, 31000, 32000, 33000, 34000, 35000, 36000, 37000, 38000, 39000, 40000, 41000, 42000, 43000, 44000, 45000, 46000, 47000, 48000, 49000, 50000, 51000, 52000, 53000, 54000, 55000, 56000, 57000, 58000, 59000, 60000, 61000, 62000, 63000, 64000, 65000, 66000, 67000, 68000, 69000, 70000, 71000, 72000, 73000, 74000, 75000, 76000, 77000, 78000, 79000, 80000, 81000, 82000, 83000, 84000, 85000, 86000, 87000, 88000, 89000, 90000, 91000, 92000, 93000, 94000, 95000, 96000, 97000, 98000, 99000, 100000, 101000, 102000, 103000, 104000, 105000, 106000, 107000, 108000, 109000, 110000, 111000, 112000, 113000, 114000, 115000, 116000, 117000, 118000, 119000, 120000, 121000, 122000, 123000, 124000, 125000, 126000, 127000, 128000, 129000, 130000, 131000, 132000, 133000, 134000, 135000, 136000, 137000, 138000, 139

In [ ]:
%%time
### main run of the function

step=100
all_inscriptions = []
for num in range(0, 3001, step):
    actual_nums = PHI[num:num+step]["PHI_ID"].tolist()
    with ThreadPoolExecutor(max_workers=step*1.5) as pool:
        currently_parsed = list(pool.map(get_inscription_data,actual_nums))
    all_inscriptions.extend(currently_parsed)
    if num in checkspoints:
      s.write_file("SDAM_data/PHI/PHI_TM_{0}.csv".format(str(num)), pd.DataFrame(all_inscriptions))
      all_inscriptions = []

Your <class 'pandas.core.frame.DataFrame'> object has been succesfully written as "https://sciencedata.dk/files/SDAM_root/SDAM_data/PHI/PHI_TM_1000.csv"
Your <class 'pandas.core.frame.DataFrame'> object has been succesfully written as "https://sciencedata.dk/files/SDAM_root/SDAM_data/PHI/PHI_TM_2000.csv"
Your <class 'pandas.core.frame.DataFrame'> object has been succesfully written as "https://sciencedata.dk/files/SDAM_root/SDAM_data/PHI/PHI_TM_3000.csv"
CPU times: user 56.3 s, sys: 4.65 s, total: 1min
Wall time: 57.2 s


In [ ]:
PHI_TM_1000 = s.read_file("SDAM_data/PHI/PHI_TM_{0}.csv".format(str(1000)))
PHI_TM_2000 = s.read_file("SDAM_data/PHI/PHI_TM_{0}.csv".format(str(2000)))
PHI_TM_3000 = s.read_file("SDAM_data/PHI/PHI_TM_{0}.csv".format(str(3000)))

In [ ]:
PHI_TM_merge = pd.DataFrame()
for df in [PHI_TM_1000, PHI_TM_2000, PHI_TM_3000]:
  PHI_TM_merge = PHI_TM_merge.append(df)

In [ ]:
len(PHI_TM_merge)

# Main extraction

In [ ]:
step=100
checkspoints = list(range(0, len(PHI), 5000))[1:] + [list(range(0, len(PHI), step))[-1]] # its own range plus last element of steps
print(checkspoints)

all_inscriptions = []
for num in range(0, len(PHI), step):
    actual_nums = PHI[num:num+step]["PHI_ID"].tolist()
    with ThreadPoolExecutor(max_workers=step*1.5) as pool:
        currently_parsed = list(pool.map(get_inscription_data,actual_nums))
    all_inscriptions.extend(currently_parsed)
    if num in checkspoints:
      s.write_file("SDAM_data/PHI/PHI_TM_{0}.csv".format(str(num)), pd.DataFrame(all_inscriptions))
      all_inscriptions = []


[5000, 10000, 15000, 20000, 25000, 30000, 35000, 40000, 45000, 50000, 55000, 60000, 65000, 70000, 75000, 80000, 85000, 90000, 95000, 100000, 105000, 110000, 115000, 120000, 125000, 130000, 135000, 140000, 145000, 150000, 155000, 160000, 165000, 170000, 175000, 180000, 185000, 190000, 195000, 200000, 205000, 210000, 215000, 218100]
Your <class 'pandas.core.frame.DataFrame'> object has been succesfully written as "https://sciencedata.dk/files/SDAM_root/SDAM_data/PHI/PHI_TM_5000.csv"
Your <class 'pandas.core.frame.DataFrame'> object has been succesfully written as "https://sciencedata.dk/files/SDAM_root/SDAM_data/PHI/PHI_TM_10000.csv"
Your <class 'pandas.core.frame.DataFrame'> object has been succesfully written as "https://sciencedata.dk/files/SDAM_root/SDAM_data/PHI/PHI_TM_15000.csv"
Your <class 'pandas.core.frame.DataFrame'> object has been succesfully written as "https://sciencedata.dk/files/SDAM_root/SDAM_data/PHI/PHI_TM_20000.csv"
Your <class 'pandas.core.frame.DataFrame'> object ha

# Merge files

In [18]:
test = s.read_file("SDAM_data/PHI/PHI_TM_{0}.csv".format(str(80000)))
test.head(20)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: DtypeWarning: Columns (5,7,89,90,108,113,119,120,121) have mixed types.Specify dtype option on import or set low_memory=False.
  """Entry point for launching an IPython kernel.


,TM_ID,EDB,EDH,EDCS,EDR,HE,UOXF,RIB,PHI,LUPA,...,ALA,CPI,GLAUx,Papyrus_Projekt,GEM,Innovating_knowledge,PUL,4CARE/DEChriM,NYU_Amheida,WARNING
0,['781347'],NaN,NaN,NaN,NaN,NaN,NaN,NaN,['200264'],NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,['781348'],NaN,NaN,NaN,NaN,NaN,NaN,NaN,['200265'],NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,['781349'],NaN,NaN,NaN,NaN,NaN,NaN,NaN,['200266'],NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,['781350'],NaN,NaN,NaN,NaN,NaN,NaN,NaN,['200267'],NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,['781351'],NaN,NaN,NaN,NaN,NaN,NaN,NaN,['200268'],NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,['781352'],NaN,NaN,NaN,NaN,NaN,NaN,NaN,['200269'],NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6,['781353'],NaN,NaN,NaN,NaN,NaN,NaN,NaN,['200270'],NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7,['781354'],NaN,NaN,NaN,NaN,NaN,NaN,NaN,['200271'],NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8,['781355'],NaN,NaN,NaN,NaN,NaN,NaN,NaN,['200272'],NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9,['781356'],NaN,NaN,NaN,NaN,NaN,NaN,NaN,['200273'],NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [19]:
step = 100
checkpoints = list(range(0, len(PHI), 5000))[1:] + [list(range(0, len(PHI), step))[-1]] # its own range plus last element of steps

PHI_TM_merged = s.read_file("SDAM_data/PHI/PHI_TM_{0}.csv".format(str(checkpoints[0])))
for num in checkspoints[1:]:
  filename = "SDAM_data/PHI/PHI_TM_{0}.csv".format(str(num))
  PHI_TM_merged = PHI_TM_merged.append(s.read_file(filename))
  print(filename)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:4: DtypeWarning: Columns (90,94,120,124,128,131,143) have mixed types.Specify dtype option on import or set low_memory=False.
  after removing the cwd from sys.path.
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:7: DtypeWarning: Columns (3,10,90,118,120,128,143) have mixed types.Specify dtype option on import or set low_memory=False.
  import sys


SDAM_data/PHI/PHI_TM_10000.csv
SDAM_data/PHI/PHI_TM_15000.csv


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:7: DtypeWarning: Columns (89,90,94,113,123,124,131,143) have mixed types.Specify dtype option on import or set low_memory=False.
  import sys


SDAM_data/PHI/PHI_TM_20000.csv


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:7: DtypeWarning: Columns (7,23,53,89,94,113,121,128,131) have mixed types.Specify dtype option on import or set low_memory=False.
  import sys


...

SDAM_data/PHI/PHI_TM_205000.csv


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:7: DtypeWarning: Columns (5,7,27,33,91,111,113,119,120,121,128,131,133) have mixed types.Specify dtype option on import or set low_memory=False.
  import sys


SDAM_data/PHI/PHI_TM_210000.csv


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:7: DtypeWarning: Columns (7,11,90,113,123,128) have mixed types.Specify dtype option on import or set low_memory=False.
  import sys


SDAM_data/PHI/PHI_TM_215000.csv
SDAM_data/PHI/PHI_TM_218100.csv


In [20]:
len(PHI_TM_merged)

218162

In [22]:
PHI_TM_merged[100000:100010]

,TM_ID,EDB,EDH,EDCS,EDR,HE,UOXF,RIB,PHI,LUPA,...,ALA,CPI,GLAUx,Papyrus_Projekt,GEM,Innovating_knowledge,PUL,4CARE/DEChriM,NYU_Amheida,WARNING
4900,['942406'],NaN,NaN,NaN,NaN,NaN,NaN,NaN,['233130'],NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4901,['942497'],NaN,NaN,NaN,NaN,NaN,NaN,NaN,['233131'],NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4902,['942497'],NaN,NaN,NaN,NaN,NaN,NaN,NaN,['233132'],NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4903,['929230'],NaN,NaN,NaN,NaN,NaN,NaN,NaN,['233133'],NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4904,['788832'],NaN,NaN,NaN,NaN,NaN,NaN,NaN,['233134'],NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4905,['788832'],NaN,NaN,NaN,NaN,NaN,NaN,NaN,['233135'],NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4906,['902850'],NaN,NaN,NaN,NaN,NaN,NaN,NaN,['233136'],NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4907,['902851'],NaN,NaN,NaN,NaN,NaN,NaN,NaN,['233137'],NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4908,['902851'],NaN,NaN,NaN,NaN,NaN,NaN,NaN,['233138'],NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4909,['942407'],NaN,NaN,NaN,NaN,NaN,NaN,NaN,['233139'],NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [29]:
s.write_file("SDAM_data/PHI/PHI_TM_merged.csv", PHI_TM_merged)

Your <class 'pandas.core.frame.DataFrame'> object has been succesfully written as "https://sciencedata.dk/files/SDAM_root/SDAM_data/PHI/PHI_TM_merged.csv"


In [40]:
#PHI_TM_merged = s.read_file("SDAM_data/PHI/PHI_TM_merged.csv")

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: DtypeWarning: Columns (2,5,7,8,10,11,12,13,20,23,25,27,29,32,33,34,53,54,55,56,66,67,70,89,90,91,94,96,99,105,107,108,111,112,113,116,118,119,120,121,122,123,124,125,126,128,131,132,133,134,135,136,141,143) have mixed types.Specify dtype option on import or set low_memory=False.
  """Entry point for launching an IPython kernel.


In [42]:
to_keep = []
for col in PHI_TM_merged.columns:
  length_vals = len([el for el in PHI_TM_merged[col] if "[" in str(el)])
  if length_vals >= 1000:
    to_keep.append(col)
    print(col, length_vals)

TM_ID 218104
EDH 8472
EDCS 23929
EDR 3761
PHI 217863
LUPA 1129
ISic 2465
IG 8147
atticinscriptions 4156
Attalus 2235


In [43]:
PHI_TM_merged = PHI_TM_merged[to_keep]
PHI_TM_merged.head(10)

,TM_ID,EDH,EDCS,EDR,PHI,LUPA,ISic,IG,atticinscriptions,Attalus
0,['786251'],NaN,NaN,NaN,['1'],NaN,NaN,['IG I³ 1'],['1672'],NaN
1,['786252'],NaN,NaN,NaN,['2'],NaN,NaN,['IG I³ 2'],['1707'],NaN
2,['786253'],NaN,NaN,NaN,['3'],NaN,NaN,['IG I³ 3'],['ii-1b'],NaN
3,['786254'],NaN,NaN,NaN,['4'],NaN,NaN,"['IG I³ 4(A)', 'IG I³ 4(B)']",['1692'],NaN
4,['786255'],NaN,NaN,NaN,['5'],NaN,NaN,['IG I³ 5'],['1284'],NaN
5,['786256'],NaN,NaN,NaN,['6'],NaN,NaN,['IG I³ 6'],['6'],NaN
6,['786257'],NaN,NaN,NaN,['7'],NaN,NaN,['IG I³ 7'],['108'],NaN
7,['786258'],NaN,NaN,NaN,['8'],NaN,NaN,['IG I³ 8'],['8'],NaN
8,['786259'],NaN,NaN,NaN,['9'],NaN,NaN,['IG I³ 9'],['9'],NaN
9,['786260'],NaN,NaN,NaN,['10'],NaN,NaN,['IG I³ 10'],['120'],NaN


In [44]:
def try_eval(val):
  if not isinstance(val, list):
    try: 
      val = eval(val)
    except:
      val = None
  return val

In [45]:
for col in PHI_TM_merged.columns:
  PHI_TM_merged[col] = PHI_TM_merged[col].apply(try_eval)

In [46]:
PHI_TM_merged.head(5)

,TM_ID,EDH,EDCS,EDR,PHI,LUPA,ISic,IG,atticinscriptions,Attalus
0,[786251],None,None,None,[1],None,None,[IG I³ 1],[1672],None
1,[786252],None,None,None,[2],None,None,[IG I³ 2],[1707],None
2,[786253],None,None,None,[3],None,None,[IG I³ 3],[ii-1b],None
3,[786254],None,None,None,[4],None,None,"[IG I³ 4(A), IG I³ 4(B)]",[1692],None
4,[786255],None,None,None,[5],None,None,[IG I³ 5],[1284],None


In [49]:
for col in PHI_TM_merged.columns:
  if "ID" not in col:
    PHI_TM_merged.rename(columns={col : col + "_ID"}, inplace=True)

In [50]:
PHI_TM_merged.head(5)

,TM_ID,EDH_ID,EDCS_ID,EDR_ID,PHI_ID,LUPA_ID,ISic_ID,IG_ID,atticinscriptions_ID,Attalus_ID
0,[786251],None,None,None,[1],None,None,[IG I³ 1],[1672],None
1,[786252],None,None,None,[2],None,None,[IG I³ 2],[1707],None
2,[786253],None,None,None,[3],None,None,[IG I³ 3],[ii-1b],None
3,[786254],None,None,None,[4],None,None,"[IG I³ 4(A), IG I³ 4(B)]",[1692],None
4,[786255],None,None,None,[5],None,None,[IG I³ 5],[1284],None


In [54]:
def get_first_id(val):
  try: 
    return int(val[0])
  except: 
    return val

PHI_TM_merged["PHI_ID"] = PHI_TM_merged["PHI_ID"].apply(get_first_id)

In [65]:
def to_int(val):
  try: 
    return int64(val)
  except: 
    return val

PHI_TM_merged["PHI_ID"] = PHI_TM_merged["PHI_ID"].apply(to_int)

In [66]:
PHI_TM_merged.head(5)

,TM_ID,EDH_ID,EDCS_ID,EDR_ID,PHI_ID,LUPA_ID,ISic_ID,IG_ID,atticinscriptions_ID,Attalus_ID
0,[786251],None,None,None,1.0,None,None,[IG I³ 1],[1672],None
1,[786252],None,None,None,2.0,None,None,[IG I³ 2],[1707],None
2,[786253],None,None,None,3.0,None,None,[IG I³ 3],[ii-1b],None
3,[786254],None,None,None,4.0,None,None,"[IG I³ 4(A), IG I³ 4(B)]",[1692],None
4,[786255],None,None,None,5.0,None,None,[IG I³ 5],[1284],None


In [64]:
PHI["PHI_ID"]

0              1
1              2
2              3
3              4
4              5
           ...  
218157    353786
218158    353787
218159    353788
218160    353789
218161    353790
Name: PHI_ID, Length: 218162, dtype: int64

In [60]:
PHI_TM_merged["PHI_ID"].astype(int)

TypeError: ignored

In [67]:
PHI.merge(PHI_TM_merged, on="PHI_ID")

,URL,Book,Text,hdr1,hdr2,tildeinfo,note,lines,metadata,data,...,link,TM_ID,EDH_ID,EDCS_ID,EDR_ID,LUPA_ID,ISic_ID,IG_ID,atticinscriptions_ID,Attalus_ID
0,/text/1?location=1701&patt=&bookid=4&offset=0&...,IG I³,1,Regions\nAttica (IG I-III),IG I³\n1,Att. — Ath.: Akr. — stoich. 35 — c. 510-500 a....,,12,1\n\n\n\n5\n\n\n\n\n10\n\n,ἔδοχσεν το͂ι δέμοι· τ̣[ὸς ἐ Σ]αλαμ̣[ῖνι κλερόχ...,...,None,[786251],None,None,None,None,None,[IG I³ 1],[1672],None
1,/text/2?location=1701&patt=&bookid=4&offset=0&...,IG I³,2,Regions\nAttica (IG I-III),IG I³\n2,Att. — non-stoich. — c. 500 a.,,14,1\n\n\n\n5\n\n\n\n\n10\n\n\n\n,[․․8-9․․․]ν̣ βολ — — — — — — — — — —\n[․6-7․․]...,...,None,[786252],None,None,None,None,None,[IG I³ 2],[1707],None
2,/text/3?location=1701&patt=&bookid=4&offset=0&...,IG I³,3,Regions\nAttica (IG I-III),IG I³\n3,Att. — stoich. 21 — 490-480 a.,,13,1\n\n\n\n5\n\n\n\n\n10\n\n\n,[․]αρ[․․․․]ι ℎερακλειο[․․5․․]\n[․]αρ̣ο#⁷[․] τι...,...,None,[786253],None,None,None,None,None,[IG I³ 3],[ii-1b],None
3,/text/4?location=1701&patt=&bookid=4&offset=0&...,IG I³,4,Regions\nAttica (IG I-III),IG I³\n4,Att. — stoich. 38 — 485/4 a.,,56,face A.1\n\n\n\n5\n\n\n\n\n10\n\n\n\n\n15\n\n\...,[․․․․․․․․․․․․․․․․․․38․․․․․․․․․․․․․․․․․․]\n[․․․...,...,None,[786254],None,None,None,None,None,"[IG I³ 4(A), IG I³ 4(B)]",[1692],None
4,/text/5?location=1701&patt=&bookid=4&offset=0&...,IG I³,5,Regions\nAttica (IG I-III),IG I³\n5,Att. — c. 500 a.,,6,1\n\n\n\n5\n,[ἔδοχσε]ν [⋮ τε͂ι βολε͂ι] ⋮ καὶ [τ]ο͂ι δέμοι ⋮...,...,None,[786255],None,None,None,None,None,[IG I³ 5],[1284],None
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
217858,/text/353786?location=1682&patt=&bookid=884&bo...,IK Strat. III,Add. et Corr. 645,Regions\nAsia Minor\nCaria,IK Strat. III\nAdd. et Corr. 645,Caria — Lagina — Rom. Imp. period? — IK Strato...,Priest inscription of Hierokles Aristophanes P...,2,6\n,[Ἀρισ]τείδης Ἱεροκλέους τοῦ Θευδώρου [— —]\n,...,http://n2t.net/ark:/99152/p08m57hqcc5,[858183],None,None,None,None,None,None,None,None
217859,/text/353787?location=1682&patt=&bookid=884&bo...,IK Strat. III,Add. et Corr. 712,Regions\nAsia Minor\nCaria,IK Strat. III\nAdd. et Corr. 712,Caria — Lagina — Rom. Imp. period? — IK Strato...,Inscription for the kleidophoros Apphion Myrta...,3,2\n\n,[Ἄπφιον] Ἀρτεμιδώ-\n[ρου] ...\n,...,http://n2t.net/ark:/99152/p08m57hqcc5,[858255],None,None,None,None,None,None,None,None
217860,/text/353788?location=1682&patt=&bookid=884&bo...,IK Strat. III,Add. et Corr. 809,Regions\nAsia Minor\nCaria,IK Strat. III\nAdd. et Corr. 809,Caria — Stratonikeia — Yeşilbağcılar — Helleni...,Sacred law (?) of Zeus Chrysaoreus. Marble ar...,2,5\n,καὶ κριὸν [— —]\n,...,http://n2t.net/ark:/99152/p03wskd55sw,[860046],None,None,None,None,None,None,None,None
217861,/text/353789?location=1682&patt=&bookid=884&bo...,IK Strat. III,Add. et Corr. 1002,Regions\nAsia Minor\nCaria,IK Strat. III\nAdd. et Corr. 1002,Caria — Stratonikeia — 276 BC — IK Stratonikei...,"Sale of a vineyard. Marble block, preserved a...",6,7\n\n\n11\n\n,"․ος Μ[αν]εω, [Πιγ?]ραμος Υσσω-\nλο[υ] ...\n......",...,None,[860064],None,None,None,None,None,None,None,None


In [69]:
s.write_file("SDAM_data/PHI/PHI_TMids.json", PHI)

Your <class 'pandas.core.frame.DataFrame'> object has been succesfully written as "https://sciencedata.dk/files/SDAM_root/SDAM_data/PHI/PHI_TMids.json"
